[View in Colaboratory](https://colab.research.google.com/github/maxmatical/pytorch-projects/blob/generative-models/DCGAN_with_GPU_Google_Colab.ipynb)

In [29]:
# install pytorch 0.4.1 with gpu

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

^C


In [30]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))


0.4.1
True
Torch 0.4.1 CUDA 8.0.61
Device: cuda:0


In [31]:
# install required packagles
!pip install torchsummary

In [0]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import save_image


from torchsummary import summary

In [0]:
# create directory

sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [0]:
# loading and preprocessing MNIST
#batchsize is 128 from paper
bs = 128
img_size = 64 # scale up imgs

transform = transforms.Compose([
                transforms.Resize(img_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=bs, 
                                          shuffle=True)

In [0]:

#Discriminator
class discriminator(nn.Module):
    def __init__(self, bs):
        super(discriminator, self).__init__()
        self.bs = bs
        self.conv1 = nn.Conv2d(1, bs, 4, stride=2, padding = 1) # no bn after conv1, input dim is 1
        self.conv2 = nn.Conv2d(bs, bs*2, 4, stride = 2, padding = 1)
        self.bn2 = nn.BatchNorm2d(bs*2) #256 channels
        self.conv3 = nn.Conv2d(bs*2, bs*4, 4, stride = 2, padding = 1)
        self.bn3 = nn.BatchNorm2d(bs*4) #512 channels
        self.conv4 = nn.Conv2d(bs*4, bs*8, 4, stride = 2, padding = 1)
        self.bn4 = nn.BatchNorm2d(bs*8) #1024 channels
        self.conv5 =  nn.Conv2d(bs*8, 1, 4, stride = 1, padding = 0)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input):
        out = self.leaky_relu(self.conv1(input))
        out = self.leaky_relu(self.bn2(self.conv2(out)))
        out = self.leaky_relu(self.bn3(self.conv3(out)))
        out = self.leaky_relu(self.bn4(self.conv4(out)))
        out = self.sigmoid(self.conv5(out))
    
        return out
    
D = discriminator(bs)
        
# Generator
class generator(nn.Module):
    def __init__(self, bs):
        super(generator, self).__init__()
        self.bs = bs
        self.deconv1 = nn.ConvTranspose2d(100, bs*8, 4, stride = 1, padding = 0) # 100 in, 1024 outchannels
        self.bn1 = nn.BatchNorm2d(bs*8) #1024 channels
        self.deconv2 = nn.ConvTranspose2d(bs*8, bs*4, 4, stride = 2, padding = 1)
        self.bn2 = nn.BatchNorm2d(bs*4) 
        self.deconv3 = nn.ConvTranspose2d(bs*4, bs*2, 4, stride = 2, padding = 1)
        self.bn3 = nn.BatchNorm2d(bs*2) 
        self.deconv4 = nn.ConvTranspose2d(bs*2, bs, 4, stride = 2, padding = 1)
        self.bn4 = nn.BatchNorm2d(bs) 
        self.deconv5 = nn.ConvTranspose2d(bs, 1, 4, stride=2, padding = 1)
        self.relu = nn.ReLU(inplace=True)
        self.tanh = nn.Tanh()
    def forward(self, input):
        out = self.relu(self.bn1(self.deconv1(input)))
        out = self.relu(self.bn2(self.deconv2(out)))
        out = self.relu(self.bn3(self.deconv3(out)))
        out = self.relu(self.bn4(self.deconv4(out)))
        out = self.tanh(self.deconv5(out))
        return out
G = generator(bs)


In [0]:
# enable GPU
use_cuda = True

In [0]:
# initialization for the networks
def init_weight(m):
    if isinstance(m, nn.Conv2d):
        nn.init.normal_(m.weight, mean=0, std=0.02)
    elif isinstance(m, nn.ConvTranspose2d):
        nn.init.normal_(m.weight, mean=0, std=0.02)
    elif type(m) == nn.BatchNorm2d:
        torch.nn.init.constant_(m.weight, 1)
        torch.nn.init.constant_(m.bias, 1)


In [0]:
if use_cuda and torch.cuda.is_available():
    G.cuda()
    D.cuda()

In [41]:
# check if models are on cuda
print(next(G.parameters()).is_cuda)
print(next(D.parameters()).is_cuda)


True
True


In [42]:
# apply initializer

G.apply(init_weight)
D.apply(init_weight)

discriminator(
  (conv1): Conv2d(1, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn4): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1))
  (leaky_relu): LeakyReLU(negative_slope=0.2, inplace)
  (sigmoid): Sigmoid()
)

In [43]:
#summary of networks
#summary(G, (100))
summary(D, (1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 32, 32]           2,176
         LeakyReLU-2          [-1, 128, 32, 32]               0
            Conv2d-3          [-1, 256, 16, 16]         524,544
       BatchNorm2d-4          [-1, 256, 16, 16]             512
         LeakyReLU-5          [-1, 256, 16, 16]               0
            Conv2d-6            [-1, 512, 8, 8]       2,097,664
       BatchNorm2d-7            [-1, 512, 8, 8]           1,024
         LeakyReLU-8            [-1, 512, 8, 8]               0
            Conv2d-9           [-1, 1024, 4, 4]       8,389,632
      BatchNorm2d-10           [-1, 1024, 4, 4]           2,048
        LeakyReLU-11           [-1, 1024, 4, 4]               0
           Conv2d-12              [-1, 1, 1, 1]          16,385
          Sigmoid-13              [-1, 1, 1, 1]               0
Total params: 11,033,985
Trainable para

In [0]:
# define loss and optimizers
criterion = nn.BCELoss()
d_opt = torch.optim.Adam(D.parameters(), lr = 0.0002, betas=(0.5, 0.999))
g_opt = torch.optim.Adam(G.parameters(), lr = 0.0002, betas=(0.5, 0.999))

In [0]:
# denorm for plotting
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)


In [49]:
# training
n_epochs = 20
total_step = len(data_loader)
for epoch in range(n_epochs):
    for i, (images, _) in enumerate(data_loader):
        
        #create labels (1 for real img, 0 for fake img)
        
        real_labels = torch.ones(bs).view(-1,1,1,1) 
        fake_labels = torch.zeros(bs).view(-1,1,1,1)
        
        #enabling cuda
        if use_cuda and torch.cuda.is_available():
          images = images.cuda()
          real_labels = real_labels.cuda()
          fake_labels = fake_labels.cuda()
        
        
        
        
        ########################
        # training discrimiator
        ########################
        
        # zeroing gradientsy
        d_opt.zero_grad()
        
        # loss for real images
        outputs = D(images).view(-1,1,1,1)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # loss for fake images
        z = torch.randn(bs, 100).view(-1, 100, 1,1) # 100 is input channels for G
        if use_cuda and torch.cuda.is_available():
          z = z.cuda()
        fake_img = G(z)
        outputs = D(fake_img).view(-1,1,1,1)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # total loss
        total_loss = d_loss_real + d_loss_fake

        total_loss.backward()
        d_opt.step()
        
        ########################
        # training generator
        ########################
        # zeroing gradients
        g_opt.zero_grad()
        
        # generating fake imgs 
        z = torch.randn(bs, 100).view(-1, 100, 1, 1)
        if use_cuda and torch.cuda.is_available():
          z = z.cuda()
        fake_img = G(z)
        outputs = D(fake_img).view(-1,1,1,1)
        
        # loss for generator
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        g_opt.step()
        
        if (i+5) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch+1, n_epochs, i+1, total_step, total_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))


# # Save the model checkpoints 
# #torch.save(G.state_dict(), 'G.ckpt')
# #torch.save(D.state_dict(), 'D.ckpt')


Epoch [1/20], Step [96/469], d_loss: 27.6310, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [1/20], Step [196/469], d_loss: 27.6310, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [1/20], Step [296/469], d_loss: 27.6310, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [1/20], Step [396/469], d_loss: 27.6310, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([128, 1, 1, 1])) that is different to the input size (torch.Size([96, 1, 1, 1])) is deprecated. Please ensure they have the same size.
  See :class:`~torch.nn.MultiLabelMarginLoss` for details.


ValueError: ignored